In [1]:
import time
startTime = int(round(time.time() * 1000))
import subprocess
import ipyparallel as ipp
def execute(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    output = ''
    # Poll process for new output until finished
    for line in iter(process.stdout.readline, ""):
        output += str(line)
        if len(line) > 0:
            print(line)
        if "successfully" in str(line):
            return 

print("Trying to shut down any active ipclusters. Ignore errors if shutdown unable due to absence of active clusters.")
execute('/home/social-sim/.local/bin/ipcluster start -n 96')

Trying to shut down any active ipclusters. Ignore errors if shutdown unable due to absence of active clusters.
b'2019-09-06 15:03:13.669 [IPClusterStart] Starting ipcluster with [daemon=False]\n'
b'2019-09-06 15:03:13.669 [IPClusterStart] Creating pid file: /home/social-sim/.ipython/profile_default/pid/ipcluster.pid\n'
b'2019-09-06 15:03:13.669 [IPClusterStart] Starting Controller with LocalControllerLauncher\n'
b'2019-09-06 15:03:14.672 [IPClusterStart] Starting 96 Engines with LocalEngineSetLauncher\n'
b'2019-09-06 15:03:54.579 [IPClusterStart] Engines appear to have started successfully\n'


In [2]:
import os
outputFile = '../output/5.3_output.csv'
out = open (outputFile, "a+")

In [3]:
"""!pip install --upgrade --user ipyparallel
!pip install --upgrade --user pynetlogo
!pip install --upgrade --user pandas
!pip install --upgrade --user numpy
!pip install --upgrade --user jpype1"""
import numpy as np
import pandas as pd
import pyNetLogo
import os
import ipyparallel
from os import environ

# Start IPCluster with 8 engines now through the command line using the following command:
# IPCluster start -n 8
# 8 engines are used for fair comparison against 8 parallel headless workspaces through NL4Py
client = ipyparallel.Client()
print(client.ids)
direct_view = client[:]
import os
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]


<AsyncResult: _push>

In [4]:
%%px 
#Jupyter notebook decorator to make the notebook cell parallelizable
import os
os.chdir(cwd) # change to the current directory to the IPCluster engines
import pyNetLogo
import pandas as pd
os.environ["netlogo_path"] = "/opt/netlogo/"
if os.environ.get('netlogo_path') is not None:
    netlogo_path = os.environ.get('netlogo_path')
else:
    netlogo_path = "C:/Program Files/NetLogo 6.0.2/"
#print(netlogo_path)
# Replace argument netlogo_home with the location of your NetLogo installation 
netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home = netlogo_path, netlogo_version = '6.0', jvmargs=["-Xmx1G","-XX:ParallelGCThreads=2"])
netlogo.load_model('../models/Wolf Sheep Predation.nlogo')

In [5]:
def simulation(runId):
    # Same netlogo commands as used for the NL4Py evaluation
    netlogo.load_model('../models/Wolf Sheep Predation.nlogo')
    netlogo.command("random-seed " + str(runId))
    netlogo.command("set initial-number-sheep 150")
    netlogo.command("set initial-number-wolves 150")
    netlogo.command("set grass-regrowth-time 50")
    netlogo.command("set sheep-gain-from-food 25")
    netlogo.command("set wolf-gain-from-food 50")
    netlogo.command("set sheep-reproduce 10")
    netlogo.command("set wolf-reproduce 10")
    netlogo.command("set show-energy? false")
    netlogo.command('set model-version "sheep-wolves-grass"')
    netlogo.command('setup')
    print(netlogo.report("ticks"))
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    counts = netlogo.repeat_report(['ticks','count sheep','count wolves'], 300)    
    print(counts)
    results = counts
    """pd.Series([counts.shape[0],
    counts.iloc[-1:1], 
    counts.iloc[-1:2]],index=['ticks','Avg. sheep', 'Avg. wolves'])"""
    print(results)
    return results   

In [6]:
import time
for runs in [2000]:
    #startTime = int(round(time.time() * 1000))
    # Run the simulations and measure execution time
    #try:
    #The following failsafe had to be added as PyNetLogo fails due to duplicated temporary log file names that are left behind from previous runs
    test = os.listdir(os.getcwd())
    for item in test:
        if item.endswith(".txt"):
            os.remove(os.path.join(os.getcwd(), item))
    ## end failsafe
    ## load balance view failed to run consistently. Therefore, reverted to using direct view in order to get PyNetLogo running on an IPCluster
    results = direct_view.map_sync(simulation, range(runs))
    #print(results)
    #except Exception as e: 
    #    print(e)
    #    pass
stopTime = int(round(time.time() * 1000))
executionTime = stopTime - startTime
print("pyNetLogo,repeatReporters," + str(runs) + "," + str(executionTime))
out.write("pyNetLogo,repeatReporters," + str(runs) + "," + str(executionTime) + "\n")
out.flush()

pyNetLogo,repeatReporters,2000,90299


In [7]:
os.system("ipcluster stop")
out.close()